# Streaming multispeaker ASR and diarization tutorial

Set your NeMo path

In [1]:
import sys
# sys.path.insert(0,'/your/path/to/NeMo/')
sys.path.insert(0,'/home/taejinp/projects/streaming_mulspk_asr/NeMo')

import nemo
print("Using Nemo PATH:", nemo.__path__[0])

# !pip install gradio==2.9.0

Using Nemo PATH: /home/taejinp/projects/streaming_mulspk_asr/NeMo/nemo


Set your NeMo path

In [2]:
from nemo.collections.asr.parts.utils.speaker_utils import audio_rttm_map
from nemo.core.config import hydra_runner
import gradio as gr
from scipy.io import wavfile
import numpy as np
import hydra
import os
import torch
from streaming_asr_and_diar import (
OnlineDiarizer,
ASR_DIAR_ONLINE
)

[NeMo W 2022-06-15 16:21:03 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-15 16:21:06 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-15 16:21:06 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Global seed set to 42


Read yaml file for online diarization. You have to specifty the following items:
    
- input manifest file (If  simulation)
- VAD model path
- Speaker embedding extractor model path
- Diarization Decoder model path (Coming soon)
- Punctuation model path (automatically download from NGC)
- Language model path (Coming soon)

Download nemo models and specify the path to config struct.

In [3]:
hydra.initialize(config_path="conf")
cfg = hydra.compose(config_name="/online_diarization_with_asr_demo_setup.yaml")
import socket
print(socket.gethostname())

if socket.gethostname() != "aiapps-06052021":
    # Please download the following models and run the code. 

    cfg.diarizer.out_dir = "./streaming_diar_output"
    os.makedirs(cfg.diarizer.out_dir, exist_ok=True)
    
    # Download CH109 dataset at: https://drive.google.com/drive/folders/1ksq10H-NZbKRfMjEP_WWyBF_G0iAJt6b?usp=sharing
    cfg.diarizer.manifest_filepath = "/your/path/to/ch109.json"

    # Download streaming VAD model at: https://drive.google.com/file/d/1ab42CaYeTkuJSMsMsMLbSS9m5e1isJzx/view?usp=sharing
    cfg.diarizer.vad.model_path = "/your/path/to/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo"

    # Download titanet-m model at: https://drive.google.com/file/d/1xAgjm0udKogPrlQF6cdHLobEKHLY9azA/view?usp=sharing
    cfg.diarizer.speaker_embeddings.model_path = "/your/path/to/titanet-m.nemo"

    # Download Conformer-CTC ASR model at: https://drive.google.com/file/d/1Xg075IbiwL0szI4_a8gYmCPaG1UsgR6E/view?usp=sharing
    cfg.diarizer.asr.model_path = "/your/path/to/Conformer-CTC-BPE_large_Riva_ASR_set_3.0_ep60.nemo"


aiapps-06052021


Initialize ASR_DIAR_ONLINE and OnlineDiarizer Class.

In [4]:
params = {}
params['use_cuda'] = True
AUDIO_RTTM_MAP = audio_rttm_map(cfg.diarizer.manifest_filepath)

diar = OnlineDiarizer(cfg)

diar.uniq_id = "en_0638"
diar.single_audio_file_path = AUDIO_RTTM_MAP[diar.uniq_id]['audio_filepath']
diar.rttm_file_path = AUDIO_RTTM_MAP[diar.uniq_id]['rttm_filepath']
# diar.rttm_file_path = None # DER calculation slows down online diarization speed
diar._init_segment_variables()

asr_diar = ASR_DIAR_ONLINE(diar, cfg=cfg.diarizer)
diar.device = asr_diar.device
asr_diar.reset()

simulation = True
# simulation = False # Run Gradio server with your microphone.

[NeMo I 2022-06-15 16:21:15 speaker_utils:81] Number of files to diarize: 109


[NeMo W 2022-06-15 16:21:15 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2022-06-15 16:21:15 features:200] PADDING: 16
[NeMo I 2022-06-15 16:21:19 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from /home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo.
[NeMo I 2022-06-15 16:21:19 clustering_diarizer:122] VAD model loaded locally from /home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo


[NeMo W 2022-06-15 16:21:20 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2022-06-15 16:21:20 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2022-06-15 16:21:20 features:200] PADDING: 16
[NeMo I 2022-06-15 16:21:20 label_models:100] loss is Angular Softmax
[NeMo I 2022-06-15 16:21:20 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/taejinp/Downloads/titanet-m.nemo.
[NeMo I 2022-06-15 16:21:20 clustering_diarizer:143] Speaker Model restored locally from /home/taejinp/Downloads/titanet-m.nemo
[NeMo I 2022-06-15 16:21:20 speaker_utils:81] Number of files to diarize: 109
[NeMo I 2022-06-15 16:21:20 streaming_asr_and_diar:360] Setting online diarization buffer to : 100
[NeMo I 2022-06-15 16:21:20 streaming_asr_and_diar:370] Setting online diarization buffer to : 150
[NeMo I 2022-06-15 16:21:20 speaker_utils:81] Number of files to diarize: 109


...

[NeMo I 2022-06-15 16:21:20 features:200] PADDING: 16
[NeMo I 2022-06-15 16:21:20 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from /home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo.
[NeMo I 2022-06-15 16:21:23 mixins:166] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-06-15 16:21:24 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/asr_datasets_prebuilt/RIVA_ASR_SET_3.0_tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    is_tarred: true
    tarred_audio_filepaths: /data/asr_datasets_prebuilt/RIVA_ASR_SET_3.0_tarred/audio__OP_0..4095_CL_.tar
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 1024
    num_workers: 16
    pin_memory: true
    
[NeMo W 2022-06-15 16:21:24 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data/Jasper_NEM

[NeMo I 2022-06-15 16:21:24 features:200] PADDING: 0
[NeMo I 2022-06-15 16:21:25 save_restore_connector:243] Model EncDecCTCModelBPE was successfully restored from /home/taejinp/gdrive/model/ASR_models/Conformer-CTC-BPE_large_Riva_ASR_set_3.0_ep60.nemo.
[NeMo I 2022-06-15 16:21:25 features:200] PADDING: 0
[NeMo I 2022-06-15 16:21:25 features:200] PADDING: 0
[NeMo I 2022-06-15 16:21:25 features:200] PADDING: 0
[NeMo I 2022-06-15 16:21:27 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmp0lm5r_8z/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-06-15 16:21:30 modelPT:215] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-06-15 16:21:30 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shuffle: true
    drop_last: false
    pin_memory: true
    num_workers: 8
    persistent_workers: true
    ds_item: punct_dataset_complete
    
[NeMo W 2022-06-15 16:21:30 m

[NeMo I 2022-06-15 16:21:32 save_restore_connector:243] Model PunctuationCapitalizationModel was successfully restored from /home/taejinp/gdrive/model/language_model/punctuation_en_distilbert.nemo.


Let's run simulated audio stream to check if streaming system is working properly. After you initiate the following function and while the function is running, you can check the transcription is being generated in realtime.  The path is ./streaming_diar_output/print_script.sh, and this can be viewed using "streaming_diarization_viewer.ipynb"


In [ ]:
from random import uniform
import time
# from rich.jupyter import jprint
import os
from IPython.display import display, clear_output
OUTPUT_DIR="/home/taejinp/projects/run_time/streaming_diar_output_univ"
i=0
while True:
    clear_output(wait=True)
    fp = open(f'{OUTPUT_DIR}/print_script.sh','r').read()
    print(str(i)+'\n'+fp)
    time.sleep(0.2)
    i += 1

In [8]:
if simulation:
    samplerate, sdata = wavfile.read(diar.single_audio_file_path)
    for index in range(int(np.floor(sdata.shape[0]/asr_diar.n_frame_len))):
        asr_diar.buffer_counter = index
        sample_audio = sdata[asr_diar.CHUNK_SIZE*(asr_diar.buffer_counter):asr_diar.CHUNK_SIZE*(asr_diar.buffer_counter+1)]
        asr_diar.callback_sim(sample_audio)
else:
    isTorch = torch.cuda.is_available()
    asr_diar.rttm_file_path = None
    iface = gr.Interface(
        fn=asr_diar.audio_queue_launcher,
        inputs=[
            gr.inputs.Audio(source="microphone", type='filepath'), 
            "state",
        ],
        outputs=[
            "textbox",
            "state",
        ],
        layout="horizontal",
        theme="huggingface",
        title=f"NeMo Streaming Conformer CTC Large - English, CUDA:{isTorch}",
        description="Demo for English speech recognition using Conformer Transducers",
        allow_flagging='never',
        live=True,
    )
    iface.launch(share=False)


Fusion embedding _emb shape: torch.Size([144, 192])
Reduced embedding emb shape: (144, 192)
p_value_stat:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2])
num _spks bincount: tensor([0, 0, 0, 0, 0, 6])
spk_counting_buffer_size: 6
emb ssize:  torch.Size([144, 192]) 6
sigs list: 53 segment_ranges: 53 segment_indexes: 53
sigs list: 60 segment_ranges: 60 segment_indexes: 60
sigs list: 70 segment_ranges: 70 segment_indexes: 70
sigs list: 83 segment_ranges: 83 segment_indexes: 83
sigs list: 144 segment_ranges: 144 segment_indexes: 144
[NeMo I 2022-06-15 16:34:20 punctuation_capitalization_model:1057] Using batch size 1 for inference
[NeMo I 2022-06-15 16:34:20 punctuation_capitalization_infer_dataset:91] Max length: 11
[NeMo I 2022-06-15 16:34:20 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-06-15 16:34:20 data_preprocessing:406] Min: 9 |                  Max: 9 |                  Mean: 9.0 |                  Median: 9.0
[NeMo I 2022-06-15 16:34:20 data_preproces

100%|██████████| 1/1 [00:00<00:00, 122.75batch/s]


AssertionError: 

Now, go to streaming_diarization_viewer.ipynb and check the realtime output.